In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from math import sqrt
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [24]:
df_orig= pd.read_csv('df_base')

In [25]:
df= pd.read_csv('cleaned_df')

In [26]:
df.head()

,Unnamed: 0,id,budget,original_language,popularity,release_date,runtime,title,revenue
0,0,1,14000000,en,6.575393,2/20/15,93.0,Hot Tub Time Machine 2,12314651
1,1,2,40000000,en,8.248895,8/6/04,113.0,The Princess Diaries 2: Royal Engagement,95149435
2,2,3,3300000,en,64.299990,10/10/14,105.0,Whiplash,13092000
3,3,4,1200000,hi,3.174936,3/9/12,122.0,Kahaani,16000000
4,4,5,0,ko,1.148070,2/5/09,118.0,Marine Boy,3923970


In [27]:
df_orig= df_orig.drop(columns=['poster_path', 'runtime', 'original_language','homepage', 'imdb_id', 'original_title', 'overview', 'status', 'Keywords', 'tagline', 'cast', 'crew', 'production_companies', 'production_countries', 'spoken_languages'])

In [28]:
df=df_orig

In [29]:
df.isnull().sum()

Unnamed: 0                  0
id                          0
belongs_to_collection    2396
budget                      0
genres                      7
popularity                  0
release_date                0
title                       0
revenue                     0
dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
Unnamed: 0               3000 non-null int64
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
popularity               3000 non-null float64
release_date             3000 non-null object
title                    3000 non-null object
revenue                  3000 non-null int64
dtypes: float64(1), int64(4), object(4)
memory usage: 211.0+ KB


In [31]:
#need to get rid of nulls and replace with '0'
df= df.fillna(0)


In [32]:
df.head()

,Unnamed: 0,id,belongs_to_collection,budget,genres,popularity,release_date,title,revenue
0,0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",6.575393,2/20/15,Hot Tub Time Machine 2,12314651
1,1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,8/6/04,The Princess Diaries 2: Royal Engagement,95149435
2,2,3,0,3300000,"[{'id': 18, 'name': 'Drama'}]",64.299990,10/10/14,Whiplash,13092000
3,3,4,0,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",3.174936,3/9/12,Kahaani,16000000
4,4,5,0,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",1.148070,2/5/09,Marine Boy,3923970


In [33]:
#All films that beloing to a collection (book, franchise, etc.) assigned a 1
df.belongs_to_collection.loc[df['belongs_to_collection']!=0]=1

In [35]:
df.head()

,Unnamed: 0,id,belongs_to_collection,budget,genres,popularity,release_date,title,revenue
0,0,1,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",6.575393,2/20/15,Hot Tub Time Machine 2,12314651
1,1,2,1,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,8/6/04,The Princess Diaries 2: Royal Engagement,95149435
2,2,3,0,3300000,"[{'id': 18, 'name': 'Drama'}]",64.299990,10/10/14,Whiplash,13092000
3,3,4,0,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",3.174936,3/9/12,Kahaani,16000000
4,4,5,0,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",1.148070,2/5/09,Marine Boy,3923970


In [36]:
df['popularity'].describe()

count    3000.000000
mean        8.463274
std        12.104000
min         0.000001
25%         4.018053
50%         7.374861
75%        10.890983
max       294.337037
Name: popularity, dtype: float64

In [37]:
#will create a feature that is profit and pct_profit
df['gross_profit']= df['revenue']- df['budget']

In [38]:
df.head()

,Unnamed: 0,id,belongs_to_collection,budget,genres,popularity,release_date,title,revenue,gross_profit
0,0,1,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",6.575393,2/20/15,Hot Tub Time Machine 2,12314651,-1685349
1,1,2,1,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,8/6/04,The Princess Diaries 2: Royal Engagement,95149435,55149435
2,2,3,0,3300000,"[{'id': 18, 'name': 'Drama'}]",64.299990,10/10/14,Whiplash,13092000,9792000
3,3,4,0,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",3.174936,3/9/12,Kahaani,16000000,14800000
4,4,5,0,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",1.148070,2/5/09,Marine Boy,3923970,3923970
